In [ ]:
import os
import numpy as np
import mne

%matplotlib widget

## Loading Data

MNE-Python data-structures are based around `FIF file format` from `Neuromag`.

EEG and MEG data from a subject performing an audiovisual experiment, along with MRI scans for that subject

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder,
    'MEG',
    'sample',
    'sample_audvis_filt-0-40_raw.fif')

In [ ]:
raw = mne.io.read_raw_fif(sample_data_raw_file)

**SSP Projector:**
A projector (abbr. proj), also referred to as Signal Space Projection (SSP), defines a linear operation applied spatially to EEG or MEG data. You can see this as a matrix multiplication that reduces the rank of the data by projecting it to a lower dimensional subspace. Such a projection operator is applied to both the data and the forward operator for source localization. Note that EEG average referencing can be done using such a projection operator. It is stored in the measurement info in info['projs'].

[Background on projectors and projections](https://mne.tools/dev/auto_tutorials/intro/plot_40_projectors_background.html#tut-projectors-background)

In [ ]:
print(raw)

In [ ]:
print(raw.info)

Power Spectral Density (PSD) and raw traces for each sensor.
For the PSD plot, we'll only plot frequencies below 50 Hz – data is low-pass filtered at 40 Hz.

In [ ]:
raw.plot_psd(fmax=50);

In [ ]:
raw.plot(duration=5, n_channels=30);

## Preprocessing

Clean up data by performing independent component analysis (ICA). We'll skip the steps that helped determine which components that best capture the artifacts.
[Repairing artifacts with ICA](https://mne.tools/dev/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#tut-artifact-ica)

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)

In [ ]:
ica.exclude = [1,2]

In [ ]:
ica.plot_properties(raw, picks=ica.exclude);

In [ ]:
orig_raw = raw.copy()
raw.load_data()

In [ ]:
ica.apply(raw)

Show some frontal channels to clearly illustrate the artifact removal

In [ ]:
chs = ['MEG 0111', 'MEG 0121', 'MEG 0131', 'MEG 0211', 'MEG 0221', 'MEG 0231',
       'MEG 0311', 'MEG 0321', 'MEG 0331', 'MEG 1511', 'MEG 1521', 'MEG 1531',
       'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006',
       'EEG 007', 'EEG 008']

chan_idxs = [raw.ch_names.index(ch) for ch in chs]
orig_raw.plot(order=chan_idxs, start=12, duration=4)
raw.plot(order=chan_idxs, start=12, duration=4);

## Detecting experimental events

In [ ]:
events = mne.find_events(raw, stim_channel='STI 014')

0: sample number
1: often ignored
2: event ID

In [ ]:
print(events[:5])

In [ ]:
event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3, 'visual/right': 4, 'smiley': 5, 'buttonpress': 32}

In [ ]:
fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw.info['sfreq']);

## Epoching continuous data

In [ ]:
reject_criteria = dict(mag=4000e-15, # 4000 fT
                      grad=4000e-13, # 4000 fT/cm
                      eeg=150e-6, # 150 uV
                      eog=250e-6 # 250 uV
                      )

In [ ]:
epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-0.2, tmax=0.5, reject=reject_criteria, preload=True)

Pool across left/right stimulus presentations so we can compare auditory vs. visual responses. To avoid biasing signals to the left or right, we'll `equalize_event_counts` to randomly sample epochs from each condition to match the number of epochs present in the condition with the fewest good epochs.

In [ ]:
conds_we_care_about = ['auditory/left', 'auditory/right', 'visual/left', 'visual/right']

In [ ]:
epochs.equalize_event_counts(conds_we_care_about) # this operates in-place

In [ ]:
aud_epochs = epochs['auditory']
vis_epochs = epochs['visual']

In [ ]:
del raw, epochs # free up memory

Show each epoch as one row of an image map, with color representing signal magnitude; the avg. evoked response and the sensor location are shown below the image

In [ ]:
aud_epochs.plot_image(picks=['MEG 1332', 'EEG 021']);

## Time-frequency analysis

* time-frequency representations
* power spectral density
* cross-spectral density

For the auditory epochs compute the induced power at different frequencies  and times using `Morlet wavelets`

In [ ]:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(aud_epochs, n_cycles=2, return_itc=False, freqs=frequencies, decim=3)

In [ ]:
power.plot(['MEG 1332']);

## Estimating evoked responses

Get estimate of evoked responses to auditory versus visual stimuli by averaging together the epochs in each condition, 

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

In [ ]:
mne.viz.plot_compare_evokeds(dict(auditory=aud_evoked, visual=vis_evoked), show_legend='upper left', show_sensors='upper right');

Examining just the EEG channels, we'll see the classic auditory evoked N100-P200 pattern over dorso-frontal electrodes, the plot scalp topographies at some additional arbitrary times:

In [ ]:
aud_evoked.plot_joint(picks='eeg');

In [ ]:
aud_evoked.plot_topomap(times=[0., 0.08, 0.1, 0.12, 0.2], ch_type='eeg');

Combining evoked responses to show contrast betwen conditions. Plot the difference wave at each sensor:

In [ ]:
evoked_diff = mne.combine_evoked([aud_evoked, -vis_evoked], weights='equal')

In [ ]:
evoked_diff.pick_types('mag').plot_topo(color='r', legend=False);

## Inverse modeling

Estimating the origins of the evoked activity by projecting the sensor data into the subject's `source space`. Here we'll use a minimum-norm estimation (MNE) to generate a continous map of activation contrained to the cortical surface. 
MNE uses a linear `inverse operator` to project EEG+MEG sensor measurements into the source space. The inverse operator is computed from the `forward solution` for this subject and estimate of `the covariance of sensor measurements`

In [ ]:
# load inverse operator
inverse_operator_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis-meg-oct-6-meg-inv.fif')
inverse_operator = mne.minimum_norm.read_inverse_operator(inverse_operator_file)

# set SNR to compute regularization parameter (λ²)
snr = 3.
lambda2 = 1. / snr ** 2

# generate the source time course (STC)
stc = mne.minimum_norm.apply_inverse(vis_evoked, inverse_operator, lambda2=lambda2, method='MNE')

In [ ]:
# path to subjects' MRI files
# subjects_dir = os.path.join(sample_data_folder, 'subjects')
# stc.plot(initial_time=0.1, hemi='split', views=['lat', 'med'], subjects_dir=subjects_dir);